Import

In [1]:
import pandas as pd
from itertools import islice
from collections import defaultdict
import networkx as nx
from itertools import combinations
import random

CSV

In [2]:
recipe_details = pd.read_csv("./culinaryDB/01_Recipe_Details.csv")
ingredients = pd.read_csv("./culinaryDB/02_Ingredients.csv")
compound_ingredients = pd.read_csv("./culinaryDB/03_Compound_Ingredients.csv")
recipe_ingredients= pd.read_csv("./culinaryDB/04_Recipe-Ingredients_Aliases.csv")


In [3]:
# 2. Raggruppa per 'Cuisine'
groups = dict(tuple(recipe_details.groupby("Cuisine")))

# 3. Rimuovi il 40% da ciascun gruppo
reduced_groups = {}

for cuisine, df in groups.items():
    reduced_df = df.sample(frac=0.3, random_state=42)  # Mantiene 60%
    reduced_groups[cuisine] = reduced_df

# 4. Ricombina tutti i gruppi ridotti in un unico DataFrame
reduced_recipe_details = pd.concat(reduced_groups.values(), ignore_index=True)

recipe_details=reduced_recipe_details
len(recipe_details)

13728

Creazione grafo

In [4]:
G = nx.Graph()

Aggiunta dei nodi ricetta

In [5]:
for _, row in recipe_details.iterrows():
    G.add_node(
        "R"+str(row['Recipe ID']),
        name=row['Title'],
        source=row['Source'],
        cuisine=row['Cuisine'],
        bipartite=0
    )

In [6]:
G.number_of_nodes()

13728

Aggiunta dei nodi archi

In [7]:
for _, row in ingredients.iterrows():
    G.add_node(
        "I"+str(row['Entity ID']),
        name=row['Aliased Ingredient Name'],
        category=row['Category'],
        bipartite=1
    )

In [8]:
G.number_of_nodes()

14658

Inserimento archi nel grafo bipartito

In [9]:
for _, row in recipe_ingredients.iterrows():
    recipe_node = f'R{row["Recipe ID"]}'
    ingredient_node = f'I{row["Entity ID"]}'

    G.add_edge(recipe_node, ingredient_node)

In [10]:
G.number_of_edges()

412560

Salvataggio Grafo bipartito

In [11]:
nx.write_graphml(G, "bipartite_graph.graphml")

Proiezione

In [12]:
ingredient_to_recipes = defaultdict(set)

for node in G.nodes:
    if G.nodes[node].get('bipartite') == 0:  # Ricetta
        for ingredient in G.neighbors(node):
            ingredient_to_recipes[ingredient].add(node)


len(ingredient_to_recipes)

597

In [ ]:
# Archi tra ricette che condividono ingredienti
edge_data = defaultdict(lambda: {"weight": 0, "shared_ingredients": set()})

for ingredient, recipes in ingredient_to_recipes.items():
    print(len(recipes))
    if 2 <= len(recipes) <= 10000:  # evita ingredienti rari e troppo comuni
        for r1, r2 in combinations(sorted(recipes), 2):
            key = (r1, r2)
            edge_data[key]["weight"] += 1
            edge_data[key]["shared_ingredients"].add(ingredient)

# Costruzione grafo
for (r1, r2), attr in edge_data.items():
    recipe_graph.add_edge(r1, r2,
                          weight=attr["weight"],
                          shared_ingredients=list(attr["shared_ingredients"]))


34
1119
4798
1630


In [104]:
recipe_graph.number_of_nodes()
recipe_graph.number_of_edges()

6784473